First, import needed modules

In [78]:
import numpy as np
import pandas as pd
import os
from functools import reduce

Initalize variables

In [79]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [80]:
import chardet #check encoding of files
with open(f'{path}/data/controls/Bevoelkerungsgruppen_Alter_2012.csv', 'rb') as file:
    print(chardet.detect(file.read()))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters/data/controls/Bevoelkerungsgruppen_Alter_2012.csv'

In [ ]:
employed = pd.read_csv(f'{path}/data/controls/Beschaeftigte_2012.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
unemployed = pd.read_csv(f'{path}/data/controls/Arbeitslose_2012.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
age_groups  = pd.read_csv(f'{path}/data/controls/Altersgruppen_2012.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
age_avg  = pd.read_csv(f'{path}/data/controls/Durchschnittsalter_2012.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
income  = pd.read_csv(f'{path}/data/controls/Einkuenfte_2010.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
area  = pd.read_csv(f'{path}/data/controls/Flaeche_2011.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
gender_nationality  = pd.read_csv(f'{path}/data/controls/Geschlecht_Nationalitaet_2011.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})
religion  = pd.read_csv(f'{path}/data/controls/Religion_2011.csv', encoding = 'ISO-8859-1', sep=';', converters = {'AGS': str})

Inspect datasets

In [ ]:
employed.head()

,AGS,Beschäftigte
0,1,973162
1,1001,27016
2,1002,80005
3,1003,69777
4,1004,25751


In [ ]:
unemployed.head()

,AGS,Arbeitslose
0,1,100048
1,1001,5127
2,1002,12732
3,1003,11009
4,1004,4376


In [ ]:
age_groups.head()

,AGS,Bevölkerung 0-17,Bevölkerung 18-24,Bevölkerung 25-44,Bevölkerung 45-64,Bevölkerung 65+
0,1,"16,7","7,7","23,5","29,9","22,1"
1,1001,"15,2","11,3","26,3","26,6","20,6"
2,1002,"14,3","11,6","30,2","25,6","18,3"
3,1003,15,"8,4","25,4","28,1","23,1"
4,1004,"16,6","8,7","23,6","28,9","22,3"


In [ ]:
age_avg.head()

,AGS,Durchschnittsalter
0,1,"44,5"
1,1001,43
2,1002,"41,8"
3,1003,"44,9"
4,1004,"44,4"


In [ ]:
income.head()

,AGS,Einkünfte je Steuerpflichtigen in Tsd.
0,1,"32,2"
1,1001,27
2,1002,"27,5"
3,1003,"27,6"
4,1004,"26,2"


In [ ]:
area.head()

,AGS,Fläche qkm
0,1,"15799,56"
1,1001,"56,74"
2,1002,"118,65"
3,1003,"214,21"
4,1004,"71,63"


In [ ]:
gender_nationality.head()

,AGS,Bevölkerung,männlich,weiblich,Ausländer/-innen
0,DG,80219695,39145941,41073754,6180013
1,1,2800119,1360484,1439635,117059
2,1001,82258,40534,41724,5129
3,1002,235782,113505,122277,16571
4,1003,210305,99832,110473,13157


In [ ]:
religion.head(10)

,AGS,Religion,Bevölkerung
0,1,Römisch-katholische Kirche,167565
1,1,Evangelische Kirche,1442840
2,1,"Sonstige, keine, ohne Angabe",1189714
3,1,Insgesamt,2800119
4,1001,Römisch-katholische Kirche,5220
5,1001,Evangelische Kirche,40867
6,1001,"Sonstige, keine, ohne Angabe",36171
7,1001,Insgesamt,82258
8,1002,Römisch-katholische Kirche,17368
9,1002,Evangelische Kirche,97355


Pivot religion table

In [ ]:
religion_pivot = pd.pivot_table(religion, index=['AGS'], columns=['Religion'], values='Bevölkerung', aggfunc=np.sum, fill_value=0)
religion_pivot.rename(columns = {'Evangelische Kirche':'Evangelisch', 'Römisch-katholische Kirche': 'Katholisch', 'Sonstige, keine, ohne Angabe': 'Rel. Sonstige/Keine'}, inplace = True)
religion_pivot = religion_pivot[['Evangelisch', 'Katholisch', 'Sonstige/Keine']]
religion_pivot.head()

Religion,Evangelisch,Katholisch,Sonstige/Keine
AGS,,,
1,1442840,167565,1189714
10,191042,619696,188885
1001,40867,5220,36171
1002,97355,17368,121059
1003,101500,17793,91012


Merge datasets

In [ ]:
dfs = [area, gender_nationality, age_avg, age_groups, employed, unemployed, income, religion_pivot]
controls = reduce(lambda  left,right: pd.merge(left,right,on=['AGS'], how='outer'), dfs)
controls.head()

,AGS,Fläche qkm,Bevölkerung,männlich,weiblich,Ausländer/-innen,Durchschnittsalter,Bevölkerung 0-17,Bevölkerung 18-24,Bevölkerung 25-44,Bevölkerung 45-64,Bevölkerung 65+,Beschäftigte,Arbeitslose,Einkünfte je Steuerpflichtigen in Tsd.,Evangelisch,Katholisch,Sonstige/Keine
0,1,"15799,56",2800119,1360484,1439635,117059,"44,5","16,7","7,7","23,5","29,9","22,1",973162,100048,"32,2",1442840,167565,1189714
1,1001,"56,74",82258,40534,41724,5129,43,"15,2","11,3","26,3","26,6","20,6",27016,5127,27,40867,5220,36171
2,1002,"118,65",235782,113505,122277,16571,"41,8","14,3","11,6","30,2","25,6","18,3",80005,12732,"27,5",97355,17368,121059
3,1003,"214,21",210305,99832,110473,13157,"44,9",15,"8,4","25,4","28,1","23,1",69777,11009,"27,6",101500,17793,91012
4,1004,"71,63",77249,37710,39539,4661,"44,4","16,6","8,7","23,6","28,9","22,3",25751,4376,"26,2",38356,5019,33874
